# Writing Data To File

In this notebook we will show how to write information to a file. CDMS2 writes out NetCDF files.

First let's read in some data


In [1]:
import os
import cdms2
home = os.path.expandvars("$HOME")
ipsl_ps_file = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ps/gr/v20180803/ps_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ps = ipsl_ps_file("ps", time=("2000", "2010"))
ps.shape

(120, 143, 144)

In [2]:
import os

Let's rewrite this smaller chunk of data into a file.

By default cdms2 use NetCDF4 Classic Format with compression:

In [3]:
print("NetCDF4:", cdms2.getNetcdf4Flag())

NetCDF4: 1


**NOTE:** cdms2 does not let you set chunking sizes, NetCDF defaults are used
These flags need to be setup before opening a netcdf file. If you prefer NetCDF3:

In [4]:
cdms2.useNetcdf3()
print("NetCDF4:", cdms2.getNetcdf4Flag())

NetCDF4: 0


In [5]:
# But let's use it and set deflate level to 5
cdms2.setNetcdf4Flag(1)
cdms2.setNetcdfDeflateFlag(1)
cdms2.setNetcdfDeflateLevelFlag(5)

We these now set, let's open a file for writing

In [6]:
fout = cdms2.open("ps_subset.nc", "w")

`"w"` means create a new file if it exists and open for writing.

Now we probably want to set a few global attributes on this file so we can remember what happened

In [7]:
import time
fout.history = "{}: Created from subset of bigger file during rgma tutorial".format(time.asctime())
fout.origin = ipsl_ps_file.name
fout.author = "me"
# And so on

Now let's write our data to this file

In [8]:
fout.write(ps)
fout.close()

/opt/conda/lib/python3.7/site-packages/cdms2/dataset.py:2193: Warning: Files are written with compression and no shuffling
You can query different values of compression using the functions:
cdms2.getNetcdfShuffleFlag() returning 1 if shuffling is enabled, 0 otherwise
cdms2.getNetcdfDeflateFlag() returning 1 if deflate is used, 0 otherwise
cdms2.getNetcdfDeflateLevelFlag() returning the level of compression for the deflate method

If you want to turn that off or set different values of compression use the functions:
value = 0
cdms2.setNetcdfShuffleFlag(value) ## where value is either 0 or 1
cdms2.setNetcdfDeflateFlag(value) ## where value is either 0 or 1
cdms2.setNetcdfDeflateLevelFlag(value) ## where value is a integer between 0 and 9 included

To produce NetCDF3 Classic files use:
cdms2.useNetCDF3()
To Force NetCDF4 output with classic format and no compressing use:
cdms2.setNetcdf4Flag(1)
NetCDF4 file with no shuffling or deflate and noclassic will be open for parallel i/o
  "for pa

If you're running this in batch mode and want to turn the warning off use:

In [9]:
cdms2.setCompressionWarnings(0)

False

Now let's look at our netcdf file

In [10]:
!ncdump -hs ps_subset.nc

netcdf ps_subset {
dimensions:
	time = UNLIMITED ; // (120 currently)
	bound = 2 ;
	lat = 143 ;
	lon = 144 ;
variables:
	double time(time) ;
		time:bounds = "time_bounds" ;
		time:axis = "T" ;
		time:calendar = "gregorian" ;
		time:units = "days since 1850-01-01 00:00:00" ;
		time:standard_name = "time" ;
		time:long_name = "Time axis" ;
		time:time_origin = "1850-01-01 00:00:00" ;
		time:realtopology = "linear" ;
		time:_Storage = "chunked" ;
		time:_ChunkSizes = 512 ;
		time:_DeflateLevel = 5 ;
		time:_Endianness = "little" ;
	double time_bounds(time, bound) ;
		time_bounds:_Storage = "chunked" ;
		time_bounds:_ChunkSizes = 1, 2 ;
		time_bounds:_DeflateLevel = 5 ;
		time_bounds:_Endianness = "little" ;
	float lat(lat) ;
		lat:bounds = "bounds_lat" ;
		lat:axis = "Y" ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:realtopology = "linear" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 143 ;
		lat:_DeflateLevel = 5 ;
		lat:_E

We can see our global attributes and compression settings are here, as well as all variable attributes.

In [11]:
extra_ps = ipsl_ps_file("ps", time=("2010", "2012"))
extra_ps.shape

(24, 143, 144)

In [12]:
fout = cdms2.open("ps_subset.nc", "r+")
fout.write(extra_ps)
fout.close()

In [13]:
!ncdump -h ps_subset.nc

netcdf ps_subset {
dimensions:
	time = UNLIMITED ; // (144 currently)
	bound = 2 ;
	lat = 143 ;
	lon = 144 ;
variables:
	double time(time) ;
		time:bounds = "time_bounds" ;
		time:axis = "T" ;
		time:calendar = "gregorian" ;
		time:units = "days since 1850-01-01 00:00:00" ;
		time:standard_name = "time" ;
		time:long_name = "Time axis" ;
		time:time_origin = "1850-01-01 00:00:00" ;
		time:realtopology = "linear" ;
	double time_bounds(time, bound) ;
	float lat(lat) ;
		lat:bounds = "bounds_lat" ;
		lat:axis = "Y" ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:realtopology = "linear" ;
	double bounds_lat(lat, bound) ;
	float lon(lon) ;
		lon:bounds = "bounds_lon" ;
		lon:axis = "X" ;
		lon:modulo = 360. ;
		lon:topology = "circular" ;
		lon:units = "degrees_east" ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:realtopology = "circular" ;
	double bounds_lon(lon, bound) ;
	float ps(time, lat, lon) ;
		ps